In [4]:
!pip3 install --user pykrx

# pykrx

[https://github.com/sharebook-kr/pykrx](https://github.com/sharebook-kr/pykrx)

# Algorithm
1. 기관상위종목 - 10일치 리스팅
2. 가장 매수가 많은 인덱스 찾기
3. 10일간 분할매수 후 30일 후 매도

In [10]:
from pykrx import stock
import pandas as pd
import numpy as np
import datetime
from dateutil import relativedelta
import warnings
import copy 
import time
import os

In [65]:
investor = ['금융투자', '보험', '투신' ,'사모', '은행']

__DBG_PRINT__ = 0
__HIGHEST__ = 10

# 코스닥기관수급
def get_kosdaq_supply(fromdate,todate,investor):
    total_dict = dict()
    for inv in investor:
        df = stock.get_market_net_purchases_of_equities_by_ticker(fromdate=fromdate, todate=todate,
            market='KOSDAQ',investor=inv).sort_values(ascending=False, by=['순매수거래대금'])
        total_dict[inv] = df
        time.sleep(1)
    return total_dict

def get_kosdaq_index_table():
    FILE_ = os.path.join('fsdata','kosdaq_index_table.csv')
    if not os.path.exists(FILE_):
        # kosdaq ticker search-테이블 //30s
        kosdaq_sector_table = pd.DataFrame([],columns=['ticker','sector','item_as_tickers','pivot'])

        for i,ticker in enumerate(stock.get_index_ticker_list(market='KOSDAQ', date='20200103')):
            data = {
                'ticker':ticker,
                'sector':stock.get_index_ticker_name(ticker),
                'item_as_tickers':[(ticker,stock.get_market_ticker_name(ticker)) for ticker in stock.get_index_portfolio_deposit_file(ticker)],
                'pivot':0
                }
            kosdaq_sector_table.loc[i] = data
            time.sleep(2)
        kosdaq_sector_table = kosdaq_sector_table.iloc[6:38]
        kosdaq_sector_table.to_csv(FILE_) 
    else:
        kosdaq_sector_table = pd.read_csv(FILE_)
        kosdaq_sector_table.set_index('Unnamed: 0',drop=True,inplace=True)
        kosdaq_sector_table.reset_index(drop=True,inplace=True)

        # string to list 필요

        # string to list 필요

        # string to list 필요

        # string to list 필요


        # string to list 필요
        # string to list 필요
        # string to list 필요

        # string to list 필요
    return kosdaq_sector_table

def vote(investor_trends,kosdaq_sector_table):
    df = investor_trends[investor[0]]
    highest10 = df.iloc[0:__HIGHEST__]

    ticker = highest10[0:__HIGHEST__].index
    if __DBG_PRINT__:
        print(highest10[0:__HIGHEST__]['종목명'])

    for i in range(__HIGHEST__):
        tgt = ticker[i]

        for idx in kosdaq_sector_table.index:
            sec = kosdaq_sector_table.loc[idx][1]
            items = kosdaq_sector_table.loc[idx][2]
            print(type(items))
            if tgt in items:
                kosdaq_sector_table.loc[idx,'pivot'] += 1
                if __DBG_PRINT__:
                    print(idx, sec)
    return kosdaq_sector_table


def get_next_date(today:datetime,days:int=1)->datetime:
    """ Return
    Next_date
    """
    next_date = today + datetime.timedelta(days=days)
    warnings.filterwarnings(action='ignore')
    while 1:
        if next_date not in stock.get_business_days(next_date.year,next_date.month):
            next_date = next_date + datetime.timedelta(days=days)
            #time.sleep(2)
        else:
            break
    return next_date

def get_prev_date(today:datetime,days:int=1)->datetime:
    """ Return
    Next_date
    """
    prev_date = today - datetime.timedelta(days=days)
    warnings.filterwarnings(action='ignore')
    while 1:
        if prev_date not in stock.get_business_days(prev_date.year,prev_date.month):
            prev_date = prev_date - datetime.timedelta(days=days)
            #time.sleep(2)
        else:
            break
    return prev_date

if __name__ == '__main__':
    kosdaq_sector_table = get_kosdaq_index_table()

    if 1:
        start_date = '2022-08-05'
        start_date = datetime.datetime.strptime(start_date,"%Y-%m-%d")
        
        fromdate = start_date
        kosdaq_sector_table['pivot'] = 0

        for i in range(3):
            todate = get_next_date(fromdate)
            print(fromdate,todate)

            #investor_trends = get_kosdaq_supply(fromdate,todate,investor)
            kosdaq_sector_table = vote(investor_trends,kosdaq_sector_table)
            if __DBG_PRINT__:
                print(kosdaq_sector_table)
            fromdate = get_next_date(todate)

2022-08-05 00:00:00 2022-08-08 00:00:00
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class '

In [45]:
kosdaq_sector_table.head(5)
    

,Unnamed: 0,ticker,sector,item_as_tickers,pivot
0,6,2024,제조업,"[('247540', '에코프로비엠'), ('028300', 'HLB'), ('06...",0
1,7,2026,건설,"[('389260', '대명에너지'), ('013120', '동원개발'), ('03...",0
2,8,2027,유통,"[('091990', '셀트리온헬스케어'), ('033290', '코웰패션'), (...",0
3,9,2029,운송,"[('003100', '선광'), ('124560', '태웅로직스'), ('3774...",0
4,10,2031,금융,"[('003380', '하림지주'), ('041190', '우리기술투자'), ('0...",0


In [58]:
reorder_table = kosdaq_sector_table.sort_values('pivot',ascending=False).iloc[1:5]
reorder_table

,ticker,sector,item_as_tickers,pivot
0,2026,건설,"[('389260', '대명에너지'), ('013120', '동원개발'), ('03...",0
1,2159,반도체,"[('058470', '리노공업'), ('005290', '동진쎄미켐'), ('10...",0
2,2158,정보기기,"[('068940', '아이씨케이'), ('347890', '엠투아이'), ('05...",0
3,2157,통신장비,"[('032500', '케이엠더블유'), ('178320', '서진시스템'), ('...",0


In [15]:
def get_score(ticker,fromdate):
    df = stock.get_market_fundamental(get_prev_date(fromdate,5),fromdate,ticker)
    time.sleep(0.5)
    if not len(df):
        score = -100
        return score
    fund = df.rolling(window=len(df)).mean().iloc[-1]
    if fund.PER <= 0:
        score = -100
        return score
    score_func = lambda x: 1/x
    score = score_func(fund.PER)
    
    return score

def get_high_score_list(sector_reorder_table,fromdate):
    __MAX_CANDIDATE__ = 5

    score_table = pd.DataFrame([],columns=['ticker','score'])

    for i,ticker in enumerate(sector_reorder_table.item_as_tickers):
        if __MAX_CANDIDATE__ > len(score_table):
            score = get_score(ticker,fromdate)
            score_table.loc[i] = [ticker,score]
        else:
            score = get_score(ticker,fromdate)
            if score_table.iloc[__MAX_CANDIDATE__-1].score > score:    
                continue
            else:
                score_table.iloc[__MAX_CANDIDATE__-1] = [ticker,score]
        score_table = score_table.sort_values(['score'],ascending=False)

    return score_table

if __name__ == '__main__':
    score_table = get_high_score_list(reorder_table.iloc[0],fromdate)

KeyboardInterrupt: 

In [ ]:
score_table

score_table['name'] = .map(score_table['ticker'],lambda x: stock.get_market_ticker_name(x))

In [ ]:

for i in range(len(score_table)):
    df = stock.get_market_ohlcv(get_prev_date(fromdate,10),get_next_date(fromdate,60),score_table.ticker[i])
    df['종가'].plot()